# Training Pipeline

# Imports
## Pip Packages

In [1]:
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

if not os.path.exists('data'):
    new_directory_path = "..\\..\\"
    os.chdir(new_directory_path)

from src.datasets import FreeViewInMemory, seq2seq_jagged_collate_fn, seq2seq_padded_collate_fn
from src.model import PathModel
import numpy as np

# Dataset

## Load Dataset

In [2]:
# TODO Implement Strategy and Builder pattern
datasetv2 = FreeViewInMemory(sample_size= 13,log = True, start_index=2)


Data loaded in memory


# Training

In [3]:
model = PathModel(input_dim = 3,
                  output_dim = 3,
                  n_encoder = 2,
                  n_decoder = 2,
                  model_dim = 256,
                  total_dim = 256,
                  n_heads = 4,
                  ff_dim = 512,
                  max_pos_enc=15,
                  max_pos_dec=26)




In [24]:


dataloader = DataLoader(datasetv2, batch_size=128, shuffle=True, num_workers=0, collate_fn= seq2seq_padded_collate_fn)
for batch in tqdm(dataloader):
    x,x_mask,y, y_mask = batch
    break
    

  0%|          | 0/274 [00:00<?, ?it/s]

tensor(True)


In [ ]:
# model = torch.compile(model)
# model(x,y)